In [1]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from timeit import default_timer as timer
from utils.config import PATH_SCENARIOS, N_REDUCED_SCNEARIOS, PATH_SCENARIOS_REDUCED
from sklearn_extra.cluster import KMedoids

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS)

In [4]:
scenarios = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'time', 'vehicle_types'])

In [5]:
X = scenarios.values

In [6]:
kmedoids = KMedoids(n_clusters=N_REDUCED_SCNEARIOS, random_state=0).fit(X)

In [7]:
# note that because the scenarios dataframe was sorted the indeces of the medoids are also the 
# scenario ids
selected_scenario_ids = kmedoids.medoid_indices_

In [8]:
selected_scenarios = scenarios.loc[selected_scenario_ids]

In [10]:
selected_scenarios = selected_scenarios.stack(['start_hex_ids', 'end_hex_ids', 'time', 'vehicle_types'])

In [11]:
os.makedirs(os.path.dirname(PATH_SCENARIOS_REDUCED), exist_ok=True)
scenarios.to_pickle(PATH_SCENARIOS_REDUCED)